In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
# criação do conjunto previsor de treino
X_train = torch.tensor([[[[4, 5, 6, 7], [5, 6, 7, 8], [8, 9, 10, 11], [4, 6, 7, 8]]], [[[-4, 5, 6, -7], [5, -6, 7, 8], [-8, 9, -10, 11], [-4, -6, -7, -8]]]]).float().to(device)

In [3]:
X_train.shape

torch.Size([2, 1, 4, 4])

In [4]:
# normalização
X_train.div_(8) # o _ indica uma operação in-place

tensor([[[[ 0.5000,  0.6250,  0.7500,  0.8750],
          [ 0.6250,  0.7500,  0.8750,  1.0000],
          [ 1.0000,  1.1250,  1.2500,  1.3750],
          [ 0.5000,  0.7500,  0.8750,  1.0000]]],


        [[[-0.5000,  0.6250,  0.7500, -0.8750],
          [ 0.6250, -0.7500,  0.8750,  1.0000],
          [-1.0000,  1.1250, -1.2500,  1.3750],
          [-0.5000, -0.7500, -0.8750, -1.0000]]]])

In [5]:
# criação do conjunto de classe de treino
y_train = torch.tensor([0, 1]).float().to(device)

In [6]:
# criação do modelo

import torch.nn as nn
from torch.optim import Adam

def get_model():
    model = nn.Sequential(
        nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3), # camada de convolução 2-D com 1 input e 1 output e kernels 3x3
        nn.MaxPool2d(kernel_size = 2), # camada de max pooling 2-D com pools 2x2
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1, 1),
        nn.Sigmoid() # por ser uma classificação binária
    ).to(device)
    
    loss_fn = nn.BCELoss() # binary cross entropy
    opt = Adam(model.parameters(), lr = 0.01)
    
    return model, loss_fn, opt

In [15]:
model, loss_fn, opt = get_model()

In [16]:
# função de treino com batch

def train_batch(x, y, model, loss_fn, opt):
    model.train() # coloca o modelo em modo de treino
    opt.zero_grad()
    predictions = model(x)
    batch_loss = loss_fn(predictions.squeeze(), y.squeeze()) # calcula o erro com os datasets sem as dimensões unitárias
    batch_loss.backward() # backpropagation, pelo grafo computacional
    opt.step() # atualização dos pesos com Adam e os gradientes calulados na linha anterior
    return batch_loss.item()

In [17]:
# criando o dataloader
from torch.utils.data import TensorDataset, DataLoader
train_dl = DataLoader(TensorDataset(X_train, y_train))

In [18]:
for epoch in range(2000):
    if (epoch % 100 == 0):
        print(epoch)
    for batch in train_dl:
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        batch_loss = train_batch(x, y, model, loss_fn, opt)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900


In [19]:
model(X_train[:1])

tensor([[0.0032]], grad_fn=<SigmoidBackward0>)